In [110]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import imutils
import os

def match_SIFT(file):
    img2 = cv2.imread("./TestWithoutRotations/images/test_image_1.png")
    file_name = file
    img1 = cv2.imread(file_name)
    img1 = cv2.pyrDown(img1)

    sift = cv2.SIFT_create(nOctaveLayers=5, nfeatures=1000, contrastThreshold=0.1)
    matcher = cv2.FlannBasedMatcher(dict(algorithm = 1, trees = 5), {})

    kp1, des1, = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)


    # bf = cv2.BFMatcher_create() #cv2.NORM_L2SQR, crossCheck=True
    matches = matcher.knnMatch(des1, des2, 2)
    matches = sorted(matches, key = lambda x : x[0].distance)

    good = [m1 for (m1,m2) in matches if m1.distance < 0.6*m2.distance]

    

    canvas = img2.copy()

    if len(good) >= 1:
        print(len(good))
        print("Enough matches found for: {}\n".format(file))
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        matchesMask = mask.ravel().tolist()
        h,w = img2.shape[:2]
        pts = np.float32([ [0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)

        dst = cv2.perspectiveTransform(pts, M)

        draw_params = dict(matchColor = (0,255, 0),
                        singlePointColor = None,
                        matchesMask = matchesMask,
                        flags =2)
        cv2.polylines(canvas, [np.int32(dst)], True, (0,0,255),3, cv2.LINE_AA)
        matched = cv2.drawMatches(img1,kp1,canvas,kp2,good,None, **draw_params)
        # img3 = cv2.drawMatchsKnn(img1,kp1,img2,kp2,good,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        # img3 = cv2.drawMatchees(img1, kp1, img2, kp2, good, None, **draw_params)
    else:
        
        print("Not enough matches are found for: {} - {}/{}".format(file ,len(good), 4))
        # draw_params = dict(matchColor = (0,255, 0),
        #                 singlePointColor = None,
        #                 matchesMask = matchesMask,
        #                 flags =2)
        # img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        # img3 = cv2.drawMatches(img1, kp1, img2, kp2, good, None, **draw_params)
    cv2.imshow("result", matched)
    cv2.waitKey()

files = os.listdir("./Training/png/")
match_SIFT("./Training/png/029-theater.png")
# for file in files:
#     file_name = "./Training/png/" + file
#     match_SIFT(file_name)
    # break


8
Enough matches found for: ./Training/png/029-theater.png

